##### DNN 기반 분류 모델 구현
- 데이터 : iris.csv
- 피처/속성 : 4개 Sepal.Length, Sepal.Width, Petal.Length, Petal.Width
- 타겟/라벨 : 3개
- 학습-방법 : 지도학습 > 분류> 다중분류
- 학습 알고리즘 : 인공신경망(ANN) -> 심층 신경망 (MLP, DNN) : 은닉층이 많은 구성
- 프레임워크 : Pytorch

[1] 모듈 로딩 및 데이터 준비 <hr>

In [1]:
# 모듈 로딩
# 모델 관련 모듈
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.classification import F1Score, MulticlassF1Score
from torchinfo import summary

# 데이터 및 시각화 관련 모듈
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [2]:
# 활용 패키지 버전 체크 ==> 사용자 정의 함수로 구현하기~~~
print(f'Pytorch v.{torch.__version__}')
print(f'Pandas v.{pd.__version__}')

Pytorch v.2.4.1
Pandas v.2.0.3


In [3]:
torch.manual_seed(1)

In [4]:
### 데이터 로딩
DATA_FILE = '../../ML/Data/iris.csv'

### CSV => DataFrame
irisDF = pd.read_csv(DATA_FILE)

### 확인
irisDF.head(1)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa


In [5]:
### 타겟 변경 => 정수화
irisDF['variety'].unique()

array(['Setosa', 'Versicolor', 'Virginica'], dtype=object)

In [6]:
labels = dict(zip(irisDF['variety'].unique().tolist(),range(3)))
print(f'labels => {labels}')

irisDF['variety'] = irisDF['variety'].replace(labels)
irisDF.head(1)

labels => {'Setosa': 0, 'Versicolor': 1, 'Virginica': 2}


,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0


[2] 모델 클래스 설계 및 정의 <hr>
- 클래스 목적 : iris 데이터를 학습 및 추론 목적
- 클래스 이름 : IrisMCFModel
- 부모 클래스 : nn.Module
- 매 개 변 수 : 층별 입출력 개수 고정하므로 필요 없음
- 속성 / 필드 :
- 기능 / 역할 : __init__() : 모델 구조 설정, forward() : 순방향 학습 <=오버라이딩(상속관계에서만 가능)
- 클래스 구조
    * 입력층 : 입력 4개(피처 개수) / 출력 10개(퍼셉트론/뉴런 개수 10개)
    * 은닉층 : 입력 10개          / 출력 5개
    * 출력층 : 입력 5개          / 출력 3개(다중 분류 : 'Setosa', 'Versicolor', 'Virginica')
- - -
- 손실함수 / 활성화 함수
    * 클래스 형태 ==> nn.MESLoss, nn.ReLU ==> __init__() 메서드
    * 함수 형태 ==> torch.nn.fuctional 아래에 ==> forward() 메서드

In [7]:
class IrisMCFModel(nn.Module):
    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()
        self.in_layer = nn.Linear(4,10)
        self.hidden_layer = nn.Linear(10,5)
        self.out_layer = nn.Linear(5,3)

    #순방향 학습 진행 메서드
    def forward(self,x) : 
        # 입력층
        y = self.in_layer(x)    # 
        y=F.relu(y)             # relu 값의 범위 : 0<=y / 시그모이드 : 0~1
        # 은닉층 : 10개 숫자의 값(>=0)
        y = self.hidden_layer(y)
        y = F.relu(y)
        # 출력층 : 5개 숫자값 / 다중 분류이므로 소프트맥스 함수 적용 => 손실함수 CrossEntropy는 소프트맥스 자동으로 진행 => 그대로 반환하기
        return self.out_layer(y)

In [8]:
model = IrisMCFModel()
print(model)

IrisMCFModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=3, bias=True)
)


In [9]:
### [테스트] 모델 사용 메모리 정보 확인
summary(model, input_size=(50000,4))

Layer (type:depth-idx)                   Output Shape              Param #
IrisMCFModel                             [50000, 3]                --
├─Linear: 1-1                            [50000, 10]               50
├─Linear: 1-2                            [50000, 5]                55
├─Linear: 1-3                            [50000, 3]                18
Total params: 123
Trainable params: 123
Non-trainable params: 0
Total mult-adds (M): 6.15
Input size (MB): 0.80
Forward/backward pass size (MB): 7.20
Params size (MB): 0.00
Estimated Total Size (MB): 8.00

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋 : iris.csv
- 피처 개수 : 4개
- 타겟 개수 : 1개
- 클래스 이름 : IrisDataset
- 부모 클래스 : utils.data.Dataset
- 속성 / 필드 : featureDF, targetDF, n_rows, n_features
- 필수 메서드
    * _ _init_ _(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    * _ _len_ _(self) : 데이터의 개수 반환
    * _ _getItem_ _(self, index) : 특정 인덱스의 피처와 타겟 반환

In [10]:
class IrisDataset(Dataset):

    def __init__(self, featureDF, targetDF):
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]

    def __len__(self) : 
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화
        feaureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)
        
        # 피처와 타겟 반환
        return feaureTS, targetTS

In [11]:
## [테스트] 데이터셋 인스턴스 생성
# 피처와 타겟 데이터 추출
featureDF = irisDF[irisDF.columns[:-1]]     # 2D (150,4)
targetDF = irisDF[irisDF.columns[-1:]]      # 2D (150,1)

# 커스텀데이터셋 인스턴스 생성
irisDS = IrisDataset(featureDF, targetDF)

# 데이터로더 인스턴스 생성
irisDL = DataLoader(irisDS)
for feature, label in irisDL:
    print(feature.shape, label.shape, feature, label)
    break

torch.Size([1, 4]) torch.Size([1, 1]) tensor([[5.1000, 3.5000, 1.4000, 0.2000]]) tensor([[0.]])


[4] 학습 준비 <hr>
- 학습 횟수 : EPOCH   <- 처음부터 끝까지 공부하는 단위
- 배치 크기 : BATCH_SIZE   <- 한번에 학습할 데이터셋 양
- 위치 지정 : DEVICE  <- 텐서 저장 및 실행 위치 (GPU/CPU)
- 학 습 율  : LR 가중치와 절편 업데이트 시 경사 하강법으로 업데이트 간격 설정 0.001~0.1 (하이퍼파라미터)

In [12]:
### 학습 진행 관련 설정
EPOCH = 1000
BATCH_SIZE = 10
BATCH_CNT = irisDF.shape[0]//BATCH_SIZE # 선택사항 - 코드에 넣을 수도 있음
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

- 인스턴스/객체 : 모델, 데이터셋, 최적화 (, 손실함수, 성능지표)

In [13]:
# 모델 인스턴스
model = IrisMCFModel()

In [14]:
### DS과 DL 인스턴스

# 학습/검증/테스트용 데이터 분리
X_train,X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)
print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')

## 학습/검증/테스트용 데이터셋
# irisDS = IrisDataset(X_train, y_train)
trainDS = IrisDataset(X_train, y_train)
valDS = IrisDataset(X_val, y_val)
testDS = IrisDataset(X_test, y_test)

# 학습용 데이터로더 인스턴스 (검증용은 필요 없음, 테스트는 양이 많을 때 개발자가 선택하여 인스턴스 생성)
# irisDL = DataLoader(irisDS, batch_size = BATCH_SIZE)
trainDL = DataLoader(trainDS, batch_size = BATCH_SIZE, drop_last=True)

(84, 4) (38, 4) (28, 4)
(84, 1) (38, 1) (28, 1)


- 최적화, 손실함수 인스턴스 생성

In [15]:
# 최적화 인스턴스 => W,b 텐서, 즉 model.parameters() 전달 - 최적화하는 이유 : 오차를 줄이기 위해서!
optimizer = optim.Adam(model.parameters(), lr=LR)

# 손실함수 인스턴스 => 분류 => 다중분류 : CrossEntropyLoss
#                            예측값은 확률값으로 전달 ==> sigmoid() AF 처리 후 전달
crossLoss = nn.CrossEntropyLoss()

In [16]:
len(trainDL), trainDL.__len__()

(8, 8)

[5] 학습 진행

In [17]:
## 학습의 효과 확인 - 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY = [[],[]], [[],[]]
CNT = len(trainDL)
print(f'CNT =>{CNT}')

for epoch in range(EPOCH):
    # 학습 모드로 모델 성정
    model.train()

    # 배치크기만큼 데이터 로딩해서 학습 진행
    loss_total, score_total = 0,0
    for featureTS, targetTS in trainDL :
        # 학습 진행
        pre_y = model(featureTS)

        # 손실계산 : CrossEntropyLoss 요구사항 : 타겟(정답)은 0차원 또는 1차원, 타입은 long 
        loss = crossLoss(pre_y, targetTS.reshape(-1).long())
        loss_total += loss.item()

        # 성능평가 계산 : MulticlassF1Score 타입은 long으로 바꾸면 안됨
        score = MulticlassF1Score(num_classes=3)(pre_y, targetTS.reshape(-1))
        # 방법2 : score = F1Score(task='multiclass', num_classes = 3)(pre_y, targetTS)
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()

    with torch.no_grad():
        # 검증용 데이터셋 생성
        val_feature_TS = torch.FloatTensor(valDS.featureDF.values)
        val_target_TS = torch.FloatTensor(valDS.targetDF.values)
        # 평가
        pre_val = model(val_feature_TS)
        # 손실 계산
        loss_val = crossLoss(pre_val, val_target_TS.reshape(-1).long())
        # 성능 평가
        score_val = MulticlassF1Score(num_classes=3)(pre_val, val_target_TS.reshape(-1))

    # 에포크 당 손실과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/CNT)
    SCORE_HISTORY[0].append(score_total/CNT)
    
    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)

    print(f'[{epoch}/{EPOCH}]\n- Train Loss : {LOSS_HISTORY[0][-1]} Score : {SCORE_HISTORY[0][-1]}')
    print(f'- Val Loss : {LOSS_HISTORY[1][-1]} Score : {SCORE_HISTORY[1][-1]}')

CNT =>8
[0/1000]
- Train Loss : 1.1644433736801147 Score : 0.14747752528637648
- Val Loss : 1.0627670288085938 Score : 0.20000000298023224
[1/1000]
- Train Loss : 1.1466117948293686 Score : 0.14747752528637648
- Val Loss : 1.056637167930603 Score : 0.20000000298023224
[2/1000]
- Train Loss : 1.1305239647626877 Score : 0.14747752528637648
- Val Loss : 1.0484484434127808 Score : 0.20000000298023224
[3/1000]
- Train Loss : 1.114012822508812 Score : 0.14747752528637648
- Val Loss : 1.0389717817306519 Score : 0.20000000298023224
[4/1000]
- Train Loss : 1.0971026867628098 Score : 0.14747752528637648
- Val Loss : 1.0284874439239502 Score : 0.20000000298023224
[5/1000]
- Train Loss : 1.0797532051801682 Score : 0.14747752528637648
- Val Loss : 1.016173243522644 Score : 0.20000000298023224
[6/1000]
- Train Loss : 1.0620037764310837 Score : 0.18864053022116423
- Val Loss : 1.002556324005127 Score : 0.31309041380882263
[7/1000]
- Train Loss : 1.0438390001654625 Score : 0.2687257193028927
- Val Los

- 테스트 할때 : no_grad() 사용

In [20]:
# 모델 검증 모드 설정
model.eval()

with torch.no_grad():
    # 검증용 데이터셋 생성
    test_featureTS = torch.FloatTensor(testDS.featureDF.values)
    test_targetTS = torch.FloatTensor(testDS.targetDF.values)
    # 추론 / 평가
    pre_test = model(test_featureTS)
    print(pre_test)
    # 손실 계산
    loss_test = crossLoss(pre_test, test_targetTS.reshape(-1).long())
    # 성능 평가
    score_test = MulticlassF1Score(num_classes=3)(pre_test, test_targetTS.reshape(-1))

tensor([[ 19.1227,   5.2893, -26.7095],
        [ -2.1508,   4.1566,  -7.3629],
        [ -6.6795,   4.8348,  -5.6214],
        [ 17.1657,   4.7537, -24.0097],
        [-12.8677,   2.0972,   1.6928],
        [ -7.9063,   4.0278,  -3.6308],
        [-13.8907,  -0.8883,   6.8884],
        [ 15.0252,   4.2275, -21.1841],
        [ 14.4032,   3.9977, -20.1989],
        [-14.9144,  -1.8676,   8.9188],
        [ -6.6377,   3.9785,  -4.0533],
        [ 16.2860,   4.5129, -22.7962],
        [-14.8486,  -0.9194,   7.5562],
        [ -7.0209,   4.5941,  -5.0224],
        [ -8.1113,   3.6588,  -2.8104],
        [ 14.6253,   4.0584, -20.5053],
        [ -5.6534,   4.3935,  -5.3431],
        [ -7.6954,   3.0892,  -2.1780],
        [ 15.0239,   4.3426, -21.4284],
        [ 15.7205,   4.3582, -22.0161],
        [ -6.7848,   3.8454,  -3.7055],
        [ -7.6540,   2.6954,  -1.5926],
        [-10.2794,   2.8680,  -0.5468],
        [ 15.9017,   4.4078, -22.2660],
        [-12.5909,   1.5983,   2.5494],


In [21]:
print(loss_test, score_test)

tensor(0.0343) tensor(0.9703)
